In [1]:
import yfinance as yf
import numpy as np
import pandas as pd

/Users/begiii/Documents/SPRING 2025/Hackathon_Spring_2025/Vortex-Sentiment-Adaptive-Volatility-VSAV-Strategy/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
tsla = yf.download('TSLA', start='2019-01-01', end='2025-03-05')
xly = yf.download('XLY', start='2019-01-01', end='2025-03-05')
spy = yf.download('SPY', start='2019-01-01', end='2025-03-05')

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [3]:
spy.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1551 entries, 2019-01-02 to 2025-03-04
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   (Close, SPY)   1551 non-null   float64
 1   (High, SPY)    1551 non-null   float64
 2   (Low, SPY)     1551 non-null   float64
 3   (Open, SPY)    1551 non-null   float64
 4   (Volume, SPY)  1551 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 72.7 KB


In [4]:
len(tsla)

1551

In [5]:
xly.head()

Price,Close,High,Low,Open,Volume
Ticker,XLY,XLY,XLY,XLY,XLY
Date,,,,,
2019-01-02,94.046104,94.574027,91.246214,91.745857,6840800
2019-01-03,92.009811,93.555877,91.849550,93.329626,6346000
2019-01-04,95.054825,95.752438,93.065677,93.254219,7269100
2019-01-07,97.204224,97.826417,95.460187,95.507318,6263100
2019-01-08,98.278908,98.806831,96.883674,98.212917,9391000


In [6]:
def calculate_vortex(df, value, n=14):
    high = df[("High", value)]
    low = df[("Low", value)]
    close = df[("Close", value)]

    # Calculate VM+ and VM-
    vm_plus = abs(high - low.shift(1))   # |Today's High - Yesterday's Low|
    vm_minus = abs(low - high.shift(1))  # |Today's Low - Yesterday's High|

    # Calculate True Range (TR)
    tr = pd.concat([
        high - low,
        abs(high - close.shift(1)),
        abs(low - close.shift(1))
    ], axis=1).max(axis=1)

    # Rolling sum for lookback period
    sum_vm_plus = vm_plus.rolling(window=n).sum()
    sum_vm_minus = vm_minus.rolling(window=n).sum()
    sum_tr = tr.rolling(window=n).sum()

    # Compute VI+ and VI-
    vi_plus = sum_vm_plus / sum_tr
    vi_minus = sum_vm_minus / sum_tr

    return vi_plus, vi_minus

In [7]:
tsla['VI+'], tsla['VI-'] = calculate_vortex(tsla, 'TSLA')
xly['VI+'], xly['VI-'] = calculate_vortex(xly, 'XLY')
spy['VI+'], spy['VI-'] = calculate_vortex(spy, 'SPY')

In [8]:
spy.head(20)

Price,Close,High,Low,Open,Volume,VI+,VI-
Ticker,SPY,SPY,SPY,SPY,SPY,,
Date,,,,,,,
2019-01-02,226.954758,227.889151,223.117450,223.144664,126925200,NaN,NaN
2019-01-03,221.538971,225.494214,221.049094,225.185768,144140700,NaN,NaN
2019-01-04,228.959625,229.612786,224.224218,224.605226,142628800,NaN,NaN
2019-01-07,230.764877,232.189121,228.324599,229.231765,103139100,NaN,NaN
2019-01-08,232.932922,233.422800,230.420083,232.978297,102512600,NaN,NaN
2019-01-09,234.021606,234.874345,232.406852,233.649665,95006600,NaN,NaN
2019-01-10,234.847076,235.101082,231.780852,232.470307,96823900,NaN,NaN
2019-01-11,234.937805,234.965019,233.168821,233.758473,73858100,NaN,NaN


In [9]:
import requests
import pandas as pd

url = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=AAPL&apikey=PNM5EHRALIOT1CKJ'

response = requests.get(url)

if response.status_code == 200:
    sentiment_data = response.json()
    sentiment_df = pd.DataFrame(sentiment_data['feed']) 
    print(sentiment_df.head())
else:
    print("API call failed:", response.status_code)


                                               title  \
0                       How to Tune Out Market Noise   
1  EXCLUSIVE: Which Magnificent 7 Stock Will Perf...   
2  New Inflation Data Dismays Bulls, CoreWeave IP...   
3  Assessing Apple's Performance Against Competit...   
4  Apple's Options Frenzy: What You Need to Know ...   

                                                 url   time_published  \
0  https://www.fool.com/investing/2025/03/28/how-...  20250328T194400   
1  https://www.benzinga.com/tech/25/03/44544129/e...  20250328T190544   
2  https://www.benzinga.com/markets/equities/25/0...  20250328T161558   
3  https://www.benzinga.com/insights/news/25/03/4...  20250328T150055   
4  https://www.benzinga.com/insights/options/25/0...  20250328T134533   

               authors                                            summary  \
0  [Motley Fool Staff]  In this podcast, Motley Fool analyst Asit Shar...   
1        [Chris Katje]  Benzinga readers pick their favorite Magnifice

In [10]:
import requests
import pandas as pd

url = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&date_from=20250101T0130&date_to=20250301T0130&limit=1000&tickers=AAPL&apikey=PNM5EHRALIOT1CKJ'

response = requests.get(url)

if response.status_code == 200:
    sentiment_data = response.json()
    sentiment_df = pd.DataFrame(sentiment_data['feed']) 
    print(sentiment_df.head())
else:
    print("API call failed:", response.status_code)

sentiment_json = response.json()
print(sentiment_json)

                                               title  \
0                       How to Tune Out Market Noise   
1  EXCLUSIVE: Which Magnificent 7 Stock Will Perf...   
2  New Inflation Data Dismays Bulls, CoreWeave IP...   
3  Assessing Apple's Performance Against Competit...   
4  Apple's Options Frenzy: What You Need to Know ...   

                                                 url   time_published  \
0  https://www.fool.com/investing/2025/03/28/how-...  20250328T194400   
1  https://www.benzinga.com/tech/25/03/44544129/e...  20250328T190544   
2  https://www.benzinga.com/markets/equities/25/0...  20250328T161558   
3  https://www.benzinga.com/insights/news/25/03/4...  20250328T150055   
4  https://www.benzinga.com/insights/options/25/0...  20250328T134533   

               authors                                            summary  \
0  [Motley Fool Staff]  In this podcast, Motley Fool analyst Asit Shar...   
1        [Chris Katje]  Benzinga readers pick their favorite Magnifice

In [11]:
sentiment_data = []
for news_item in sentiment_json.get("feed", []):
    sentiment_data.append({
            "time_published": pd.to_datetime(news_item["time_published"]),
            "sentiment_score": news_item["overall_sentiment_score"],
            "sentiment_label": news_item["overall_sentiment_label"],
    })
sentiment_data = pd.DataFrame(sentiment_data)

In [12]:
sentiment_data['time_published'] = sentiment_data['time_published'].dt.date

In [13]:
pd.to_datetime(sentiment_data['time_published'])

0     2025-03-28
1     2025-03-28
2     2025-03-28
3     2025-03-28
4     2025-03-28
         ...    
669   2025-02-04
670   2025-02-04
671   2025-02-04
672   2025-02-04
673   2025-02-04
Name: time_published, Length: 674, dtype: datetime64[ns]

In [14]:
sentiment_scores_filtered = sentiment_data[pd.to_datetime(sentiment_data['time_published']).isin(tsla.index)]
sentiment_scores_filtered = sentiment_scores_filtered.groupby('time_published')['sentiment_score'].mean().reset_index()

In [ ]:
# Fix the multi-level column issue by selecting the 'Volume' column and resetting its name
tsla_volume = tsla[('Volume', 'TSLA')].rename('Volume')

# Ensure the index of tsla_volume is a column and convert it to match the type of time_published
tsla_volume = tsla_volume.reset_index()
tsla_volume['Date'] = pd.to_datetime(tsla_volume['Date'])

# Convert time_published to datetime
sentiment_scores_filtered['time_published'] = pd.to_datetime(sentiment_scores_filtered['time_published'])

# Merge the dataframes
merged_data = pd.merge(tsla_volume, sentiment_scores_filtered, left_on='Date', right_on='time_published', how='inner')

In [16]:
merged_data
merged_data['Weighted_Sentiment'] = merged_data['Volume'] * merged_data['sentiment_score']
merged_data['5_day_avg_sentiment'] = merged_data['Weighted_Sentiment'].rolling(window=5).mean()
merged_data['Buy_Condition'] = merged_data['5_day_avg_sentiment'] > 0

merged_data['5_day_avg_sentiment_norm'] = merged_data['5_day_avg_sentiment']/merged_data['Volume'].mean()

In [17]:
#merged_data_daily = merged_data.groupby('Date')['Weighted_Sentiment'].sum().reset_index()

In [18]:
sentiment_df

,title,url,time_published,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment
0,How to Tune Out Market Noise,https://www.fool.com/investing/2025/03/28/how-...,20250328T194400,[Motley Fool Staff],"In this podcast, Motley Fool analyst Asit Shar...",https://g.foolcdn.com/editorial/images/812797/...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Technology', 'relevance_score': '0...",0.071763,Neutral,"[{'ticker': 'DKNG', 'relevance_score': '0.0076..."
1,EXCLUSIVE: Which Magnificent 7 Stock Will Perf...,https://www.benzinga.com/tech/25/03/44544129/e...,20250328T190544,[Chris Katje],Benzinga readers pick their favorite Magnifice...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Trading,www.benzinga.com,"[{'topic': 'Retail & Wholesale', 'relevance_sc...",0.266216,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.1984..."
2,"New Inflation Data Dismays Bulls, CoreWeave IP...",https://www.benzinga.com/markets/equities/25/0...,20250328T161558,[The Arora Report],"To gain an edge, this is what you need to know...",https://thearorareport.com/wp-content/uploads/...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Technology', 'relevance_score': '0...",0.240809,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.1725..."
3,Assessing Apple's Performance Against Competit...,https://www.benzinga.com/insights/news/25/03/4...,20250328T150055,[Benzinga Insights],Amidst the fast-paced and highly competitive b...,https://www.benzinga.com/files/images/story/20...,Benzinga,Trading,www.benzinga.com,"[{'topic': 'Earnings', 'relevance_score': '0.8...",0.265078,Somewhat-Bullish,"[{'ticker': 'AAPL', 'relevance_score': '0.4680..."
4,Apple's Options Frenzy: What You Need to Know ...,https://www.benzinga.com/insights/options/25/0...,20250328T134533,[Benzinga Insights],Investors with a lot of money to spend have ta...,https://www.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.185008,Somewhat-Bullish,"[{'ticker': 'EVR', 'relevance_score': '0.06568..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
669,Apple Will Probably Lose Its Market Cap Crown ...,https://www.fool.com/investing/2025/02/04/appl...,20250204T165500,[Rick Munarriz],The consumer tech giant is on top with its $3....,https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",0.136503,Neutral,"[{'ticker': 'AAPL', 'relevance_score': '0.6784..."
670,"Trump's Tariffs Hit TSLA Hard, Shares Fall 5%:...",https://www.zacks.com/stock/news/2409358/trump...,20250204T155300,[Rimmi Singhi],Tesla becomes one of the S&P 500's worst perfo...,https://staticx-tuner.zacks.com/images/article...,Zacks Commentary,n/a,www.zacks.com,"[{'topic': 'Retail & Wholesale', 'relevance_sc...",0.131369,Neutral,"[{'ticker': 'MSFT', 'relevance_score': '0.0646..."
671,"Fujifilm launches hybrid instant camera ""insta...",https://markets.businessinsider.com/news/stock...,20250204T154441,[Plentisoft],"Fujifilm launches hybrid instant camera ""insta...",https://asset.fujifilm.com/www/jp/files/2025-0...,Business Insider,RSS,markets.businessinsider.com,"[{'topic': 'Technology', 'relevance_score': '1...",0.313547,Somewhat-Bullish,"[{'ticker': 'GOOG', 'relevance_score': '0.0878..."
672,"DeepSeek Disrupts, Big Tech Responds",https://www.fool.com/investing/2025/02/04/deep...,20250204T153300,[Motley Fool Staff],The market was left with more questions than a...,https://g.foolcdn.com/editorial/images/806201/...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",0.188005,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.0742..."


## volatility

In [19]:

# Flatten MultiIndex columns 
tsla.columns = [
    '_'.join(col).strip() if isinstance(col, tuple) else col
    for col in tsla.columns
]

# Calculate True Range
tsla["prev_close"] = tsla["Close_TSLA"].shift(1)
tsla["tr1"] = tsla["High_TSLA"] - tsla["Low_TSLA"]
tsla["tr2"] = abs(tsla["High_TSLA"] - tsla["prev_close"])
tsla["tr3"] = abs(tsla["Low_TSLA"] - tsla["prev_close"])

tsla["true_range"] = tsla[["tr1", "tr2", "tr3"]].max(axis=1)

# 10-day ATR
tsla["ATR_10"] = tsla["true_range"].rolling(window=10).mean()

# ---- STEP 4: Calculate ATR as a percentage of closing price ----
tsla["atr_pct"] = tsla["ATR_10"] / tsla["Close_TSLA"]

# allocating the capital

def position_size(row):
    if row["atr_pct"] < 0.03:  # < 3% volatility → low risk
        return 0.01  # allocate 1% of capital
    else:  # ≥ 3% volatility → high risk
        return 0.005  # allocate 0.5% of capital

tsla["position_size"] = tsla.apply(position_size, axis=1)

# ---- STEP 6: Optional - Capital allocation per trade ----
#capital = 100000 # Example: $100K total portfolio
#tsla["allocation_dollars"] = tsla["position_size"] * capital

# ---- Preview ----
print(tsla[["Close_TSLA", "ATR_10", "atr_pct", "position_size"]].tail(10))


            Close_TSLA     ATR_10   atr_pct  position_size
Date                                                      
2025-02-19  360.559998  16.703000  0.046325          0.005
2025-02-20  354.399994  16.464999  0.046459          0.005
2025-02-21  337.799988  17.021997  0.050391          0.005
2025-02-24  330.529999  16.770996  0.050740          0.005
2025-02-25  302.799988  18.879996  0.062351          0.005
2025-02-26  290.799988  18.412994  0.063318          0.005
2025-02-27  281.950012  18.257996  0.064756          0.005
2025-02-28  292.980011  18.067996  0.061670          0.005
2025-03-03  284.649994  19.281998  0.067739          0.005
2025-03-04  272.040009  20.654996  0.075926          0.005


In [20]:
import plotly.express as px
fig = px.line(tsla, x=tsla.index, y="atr_pct", title="ATR% Over Time")
fig.add_hline(y=0.03, line_dash="dot", line_color="green", annotation_text="Low Volatility Cutoff")
fig.show()


In [21]:
import plotly.express as px

# Filter only 2025 data
tsla_2025 = tsla[tsla.index.year == 2025]

# Plot
fig = px.line(tsla_2025, x=tsla_2025.index, y="atr_pct", title="ATR% Over Time (2025 Only)")
fig.add_hline(y=0.03, line_dash="dot", line_color="green", annotation_text="Low Volatility Cutoff")
fig.show()


In [22]:
print(tsla.columns)


Index(['Close_TSLA', 'High_TSLA', 'Low_TSLA', 'Open_TSLA', 'Volume_TSLA',
       'VI+_', 'VI-_', 'prev_close', 'tr1', 'tr2', 'tr3', 'true_range',
       'ATR_10', 'atr_pct', 'position_size'],
      dtype='object')


In [23]:
# Calculate ATR percentage
tsla['atr_pct'] = tsla['ATR_10'] / tsla['Close_TSLA']

# Create Buy Signal (assuming VI_Cross_Up is defined elsewhere)
tsla['Buy_Signal'] = tsla['VI+_'] > tsla['VI-_']  # Vortex crossover
# + add any other buy conditions here...

# Create Sell Signal (basic)
tsla['Sell_Signal'] = tsla['VI-_'] > tsla['VI+_']

# Initialize position state
tsla['Position'] = 0
peak_price = 0

for i in range(1, len(tsla)):
    if tsla['Buy_Signal'].iloc[i]:
        tsla.at[tsla.index[i], 'Position'] = 1
        peak_price = tsla['Close_TSLA'].iloc[i]
    elif tsla['Position'].iloc[i - 1] == 1:
        current_price = tsla['Close_TSLA'].iloc[i]
        peak_price = max(peak_price, current_price)
        drawdown = (peak_price - current_price) / peak_price

        if drawdown >= 0.03:
            tsla.at[tsla.index[i], 'Sell_Signal'] = True  # trailing stop
            tsla.at[tsla.index[i], 'Position'] = 0
        else:
            tsla.at[tsla.index[i], 'Position'] = 1


In [24]:
print("Buy signals:", tsla['Buy_Signal'].sum())
print("Sell signals:", tsla['Sell_Signal'].sum())


Buy signals: 857
Sell signals: 680


In [25]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=tsla.index, y=tsla['Close_TSLA'], mode='lines', name='TSLA Price'))

# Buy markers
fig.add_trace(go.Scatter(
    x=tsla[tsla['Buy_Signal']].index,
    y=tsla[tsla['Buy_Signal']]['Close_TSLA'],
    mode='markers',
    marker=dict(symbol='triangle-up', size=10, color='green'),
    name='Buy Signal'
))

# Sell markers
fig.add_trace(go.Scatter(
    x=tsla[tsla['Sell_Signal']].index,
    y=tsla[tsla['Sell_Signal']]['Close_TSLA'],
    mode='markers',
    marker=dict(symbol='triangle-down', size=10, color='red'),
    name='Sell Signal'
))

fig.update_layout(title='TSLA Buy & Sell Signals', template='plotly_white')
fig.show()


In [26]:
# Calculate ATR percentage
tsla['atr_pct'] = tsla['ATR_10'] / tsla['Close_TSLA']

# Vortex crossover logic
tsla['VI_Cross_Up'] = (tsla['VI+_'] > tsla['VI-_']) & (tsla['VI+_'].shift(1) <= tsla['VI-_'].shift(1))
tsla['VI_Cross_Down'] = (tsla['VI-_'] > tsla['VI+_']) & (tsla['VI-_'].shift(1) <= tsla['VI+_'].shift(1))

# Initialize signal & state columns
tsla['Buy_Signal'] = False
tsla['Sell_Signal'] = False
tsla['Position'] = 0
tsla['Entry_Type'] = None  # aggressive/conservative

# Trailing stop logic variables
in_position = False
peak_price = 0

for i in range(1, len(tsla)):
    row = tsla.iloc[i]
    idx = tsla.index[i]

    # Buy condition
    if not in_position and row['VI_Cross_Up']:
        tsla.at[idx, 'Buy_Signal'] = True
        tsla.at[idx, 'Position'] = 1
        in_position = True
        peak_price = row['Close_TSLA']

        # Entry Type: aggressive if ATR < 3%, else conservative
        if row['atr_pct'] < 0.03:
            tsla.at[idx, 'Entry_Type'] = 'aggressive'
        else:
            tsla.at[idx, 'Entry_Type'] = 'conservative'

    # While in position, check for trailing stop or VI cross down
    elif in_position:
        current_price = row['Close_TSLA']
        peak_price = max(peak_price, current_price)
        drawdown = (peak_price - current_price) / peak_price

        if drawdown >= 0.03 or row['VI_Cross_Down']:
            tsla.at[idx, 'Sell_Signal'] = True
            tsla.at[idx, 'Position'] = 0
            in_position = False
        else:
            tsla.at[idx, 'Position'] = 1

# Show result counts
print("Buy signals:", tsla['Buy_Signal'].sum())
print("Sell signals:", tsla['Sell_Signal'].sum())
print("Aggressive entries:", (tsla['Entry_Type'] == 'aggressive').sum())
print("Conservative entries:", (tsla['Entry_Type'] == 'conservative').sum())


Buy signals: 80
Sell signals: 80
Aggressive entries: 5
Conservative entries: 75


In [27]:
import plotly.graph_objects as go

fig = go.Figure()

# Plot TSLA closing price
fig.add_trace(go.Scatter(
    x=tsla.index, 
    y=tsla['Close_TSLA'], 
    mode='lines', 
    name='TSLA Price', 
    line=dict(color='blue')
))

# Aggressive buys
fig.add_trace(go.Scatter(
    x=tsla[(tsla['Buy_Signal']) & (tsla['Entry_Type'] == 'aggressive')].index,
    y=tsla[(tsla['Buy_Signal']) & (tsla['Entry_Type'] == 'aggressive')]['Close_TSLA'],
    mode='markers',
    name='Buy (Aggressive)',
    marker=dict(symbol='triangle-up', color='limegreen', size=10)
))

# Conservative buys
fig.add_trace(go.Scatter(
    x=tsla[(tsla['Buy_Signal']) & (tsla['Entry_Type'] == 'conservative')].index,
    y=tsla[(tsla['Buy_Signal']) & (tsla['Entry_Type'] == 'conservative')]['Close_TSLA'],
    mode='markers',
    name='Buy (Conservative)',
    marker=dict(symbol='triangle-up', color='green', size=10)
))

# Sells
fig.add_trace(go.Scatter(
    x=tsla[tsla['Sell_Signal']].index,
    y=tsla[tsla['Sell_Signal']]['Close_TSLA'],
    mode='markers',
    name='Sell Signal',
    marker=dict(symbol='triangle-down', color='red', size=10)
))

fig.update_layout(
    title='TSLA Buy/Sell Signals Over Time',
    xaxis_title='Date',
    yaxis_title='Price (USD)',
    template='plotly_white',
    height=600
)

fig.show()


In [28]:
print(merged_data.columns)

Index(['Date', 'Volume', 'time_published', 'sentiment_score',
       'Weighted_Sentiment', '5_day_avg_sentiment', 'Buy_Condition',
       '5_day_avg_sentiment_norm'],
      dtype='object')


In [29]:
tsla_signals = tsla.reset_index()[['Date', 'VI_Cross_Up', 'VI_Cross_Down', 'atr_pct']]


In [30]:
merged_data = pd.merge(merged_data, tsla_signals, on='Date', how='left')


In [31]:
merged_data

,Date,Volume,time_published,sentiment_score,Weighted_Sentiment,5_day_avg_sentiment,Buy_Condition,5_day_avg_sentiment_norm,VI_Cross_Up,VI_Cross_Down,atr_pct
0,2025-02-04,57072200,2025-02-04,0.242685,1.385054e+07,NaN,False,NaN,False,True,0.044573
1,2025-02-05,57223300,2025-02-05,0.222392,1.272602e+07,NaN,False,NaN,False,False,0.047093
2,2025-02-06,77918200,2025-02-06,0.245102,1.909789e+07,NaN,False,NaN,False,False,0.048434
3,2025-02-07,70298300,2025-02-07,0.251546,1.768323e+07,NaN,False,NaN,False,False,0.052102
4,2025-02-10,77514900,2025-02-10,0.246183,1.908285e+07,1.648810e+07,True,0.190556,False,False,0.052151
5,2025-02-11,118543400,2025-02-11,0.172520,2.045115e+07,1.780823e+07,True,0.205813,False,False,0.059193
6,2025-02-12,105382700,2025-02-12,0.187009,1.970755e+07,1.920453e+07,True,0.221950,False,False,0.058911
7,2025-02-13,89441500,2025-02-13,0.186973,1.672312e+07,1.872958e+07,True,0.216461,False,False,0.054034
8,2025-02-14,68277300,2025-02-14,0.192267,1.312748e+07,1.781843e+07,True,0.205931,False,False,0.052585
9,2025-02-18,51631700,2025-02-18,0.195193,1.007816e+07,1.601749e+07,True,0.185117,False,False,0.046867


In [32]:
strategy_data = merged_data.copy()  # important to ensure safe assignment

strategy_data.loc[:, 'Buy_Signal'] = (
    (strategy_data['5_day_avg_sentiment'] > 0) &
    (strategy_data['atr_pct'] > 0)
)

strategy_data.loc[:, 'Entry_Type'] = strategy_data['atr_pct'].apply(
    lambda x: 'Aggressive' if x < 0.03 else 'Conservative'
)


In [33]:
print("Buy signals (w/ sentiment + VI + ATR):", strategy_data['Buy_Signal'].sum())
print(strategy_data[['Date', 'Buy_Signal', '5_day_avg_sentiment', 'atr_pct']].tail(10))


Buy signals (w/ sentiment + VI + ATR): 16
         Date  Buy_Signal  5_day_avg_sentiment   atr_pct
10 2025-02-19        True         1.431937e+07  0.046325
11 2025-02-20        True         1.209762e+07  0.046459
12 2025-02-21        True         1.160231e+07  0.050391
13 2025-02-24        True         1.204331e+07  0.050740
14 2025-02-25        True         1.354197e+07  0.062351
15 2025-02-26        True         1.448089e+07  0.063318
16 2025-02-27        True         1.632379e+07  0.064756
17 2025-02-28        True         1.784810e+07  0.061670
18 2025-03-03        True         1.944193e+07  0.067739
19 2025-03-04        True         2.094401e+07  0.075926


In [34]:
import plotly.graph_objects as go

# Ensure 'Date' is datetime and set as index if needed
strategy_data['Date'] = pd.to_datetime(strategy_data['Date'])

fig = go.Figure()

# Plot 5-day Avg Sentiment
fig.add_trace(go.Scatter(
    x=strategy_data['Date'],
    y=strategy_data['5_day_avg_sentiment_norm'],
    mode='lines+markers',
    name='5-Day Avg Sentiment',
    line=dict(color='blue')
))

# Plot ATR %
fig.add_trace(go.Scatter(
    x=strategy_data['Date'],
    y=strategy_data['atr_pct'],
    mode='lines+markers',
    name='ATR %',
    yaxis='y2',
    line=dict(color='orange')
))

# Optional: Highlight Buy Signal Dates (even though there are none now)
fig.add_trace(go.Scatter(
    x=strategy_data.loc[strategy_data['Buy_Signal'], 'Date'],
    y=strategy_data.loc[strategy_data['Buy_Signal'], '5_day_avg_sentiment_norm'],
    mode='markers',
    marker=dict(color='green', size=10, symbol='star'),
    name='Buy Signal'
))

# Add dual axis layout
fig.update_layout(
    title="5-Day Sentiment vs ATR % (with Buy Signals)",
    xaxis_title='Date',
    yaxis=dict(title='5-Day Avg Sentiment'),
    yaxis2=dict(title='ATR %', overlaying='y', side='right'),
    legend=dict(x=0.01, y=0.99),
    height=500
)

fig.show()
